## 1. Setup and Imports

In [ ]:
import sys
import os

# Add parent directory to path
sys.path.append('..')

from src.orchestrator import Orchestrator
from src.tools import PatientDataTool, SearchTool, AnalysisTool, ReportTool

print("Imports successful!")

## 2. Initialize the Orchestrator

The orchestrator coordinates all tools and agents.

In [ ]:
# Initialize the orchestrator with patient data path
orchestrator = Orchestrator(data_path="../data/patients.json")

print("\nOrchestrator initialized with:")
print(f"- Data Tool: {orchestrator.data_tool.__class__.__name__}")
print(f"- Agents: {orchestrator.coordinator.name}")
print(f"- Memory: {orchestrator.long_term_memory.__class__.__name__}")

## 3. Load and Inspect Patient Data

In [ ]:
# Get all patients
patients = orchestrator.data_tool.get_all_patients()

print(f"Total patients loaded: {len(patients)}")
print("\nSample patient data:")
for patient in patients[:3]:  # Show first 3 patients
    print(f"  ID: {patient.get('id')}, Name: {patient.get('name')}, Age: {patient.get('age')}")

## 4. Query Examples

### 4.1 Find Specific Patient

In [ ]:
# Find patient by ID
response = orchestrator.run("Find patient P001")
print("Query: Find patient P001")
print(f"Response: {response}")

### 4.2 Get All Patients

In [ ]:
# Get all patients
response = orchestrator.run("Get all patients")
print("Query: Get all patients")
print(f"Response: {response}")

### 4.3 Search Patients

In [ ]:
# Search for patients
response = orchestrator.run("Search for Sample")
print("Query: Search for Sample")
print(f"Response: {response}")

## 5. Statistical Analysis

In [ ]:
# Get statistics
stats = orchestrator.get_statistics()

print("Patient Statistics:")
print(f"  Total Patients: {stats.get('total_patients', 0)}")
print(f"  Average Age: {stats.get('average_age', 0):.1f}")
print(f"  Age Range: {stats.get('min_age', 0)} - {stats.get('max_age', 0)}")

## 6. Generate Report

In [ ]:
# Generate comprehensive report
report = orchestrator.generate_report()
print(report)

## 7. Agent Interaction Examples

### 7.1 Data Retrieval Agent

In [ ]:
# Direct interaction with data retrieval agent
response = orchestrator.data_agent.process("Get patient P001")
print("Data Retrieval Agent:")
print(f"Response: {response}")

### 7.2 Analysis Agent

In [ ]:
# Direct interaction with analysis agent
response = orchestrator.analysis_agent.process("Calculate statistics")
print("Analysis Agent:")
print(f"Response: {response}")

## 8. Memory Inspection

View the conversation history stored in memory.

In [ ]:
# Check long-term memory
print(f"Total interactions in memory: {len(orchestrator.long_term_memory)}")
print("\nRecent interactions:")
for entry in orchestrator.long_term_memory.get_recent(5):
    print(f"  [{entry['role']}]: {entry['content'][:50]}...")

## 9. Custom Queries

Try your own queries with the orchestrator.

In [ ]:
# Custom query
query = "Your query here"
response = orchestrator.run(query)
print(f"Query: {query}")
print(f"Response: {response}")

## 10. Summary

This notebook demonstrated:
- ✅ Orchestrator initialization
- ✅ Patient data loading and inspection
- ✅ Query processing through agents
- ✅ Statistical analysis
- ✅ Report generation
- ✅ Memory management

The PDAA Agent system successfully coordinates multiple specialized agents to handle patient data analysis tasks.

# Post-Discharge Adherence Agent (PDAA) — Step-by-Step Guide
This notebook follows a clear plan to build and run a 3-agent system (Monitor, Analyzer, Escalator) with memory and tools, and simulate a 7-day loop per patient.

## Step 1 — Setup
- Install: `pip install google-generativeai pandas python-dotenv jupyter`
- Add `.env` with `GEMINI_API_KEY`.
- Import orchestrator and run a quick smoke test below.

In [ ]:
# Quick smoke test: import and run orchestrator for 1 day
from src.orchestrator import PDAAOrchestrator
orc = PDAAOrchestrator()
_ = orc.run_simulation(days=1)

## Step 2 — Dataset
Dataset lives in `data/patients.json`. Edit inline below if needed.

## Step 3 — Visualize Results
Load `simulation_results.json` and explore adherence trends and escalations.

In [ ]:
import json
import pandas as pd
from pathlib import Path

# Load results
results_path = Path('../simulation_results.json') if not Path('simulation_results.json').exists() else Path('simulation_results.json')
with open(results_path, 'r') as f:
    results = json.load(f)

# Build patient-level adherence DataFrame
rows = []
for pid, pres in results['patient_results'].items():
    for day_entry in pres['daily_results']:
        rows.append({
            'patient_id': pid,
            'patient_name': pres['patient_name'],
            'day': day_entry['day'],
            'score': day_entry['analysis']['adherence_score']['total_score'],
            'risk': day_entry['analysis']['risk_assessment']['risk_class'],
            'escalated': day_entry['escalation']['escalated']
        })
df = pd.DataFrame(rows)
df.head()

In [ ]:
# Summary stats per patient
summary = df.groupby(['patient_id','patient_name']).agg(
    avg_score=('score','mean'),
    escalations=('escalated','sum')
).reset_index()
summary.sort_values('avg_score', ascending=False)

In [ ]:
# Risk count by day
risk_counts = df.groupby(['day','risk']).size().reset_index(name='count')
risk_counts.head()

## Step 4 — Edit Patients Inline
Load, edit, and save `data/patients.json` from the notebook for quick tweaks.

In [ ]:
import json
from pathlib import Path

patients_path = Path('../data/patients.json') if not Path('data/patients.json').exists() else Path('data/patients.json')
with open(patients_path, 'r') as f:
    patients = json.load(f)
patients

In [ ]:
# Example tweak: change follow_up of first patient
patients[0]['discharge_plan']['follow_up'] = '2025-12-01'
with open(patients_path, 'w') as f:
    json.dump(patients, f, indent=2)
print('Saved updated patients.json')

## Step 5 — Re-run Simulation
Reload orchestrator and run with updated data to regenerate results.

In [ ]:
from src.orchestrator import PDAAOrchestrator
orc = PDAAOrchestrator(patients_file=str(patients_path))
results = orc.run_simulation(days=3)
orc.export_results(results)

## Step 6 — Charts
Visualize adherence trends and escalations to spot patterns quickly.

In [ ]:
# Line chart: adherence score over days per patient
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
for pid, group in df.groupby('patient_id'):
    plt.plot(group['day'], group['score'], marker='o', label=group['patient_name'].iloc[0])
plt.title('Adherence Score Over Days')
plt.xlabel('Day')
plt.ylabel('Score (0-100)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Bar chart: escalations per patient
esc_counts = df.groupby('patient_name')['escalated'].sum().reset_index()
plt.figure(figsize=(8,5))
plt.bar(esc_counts['patient_name'], esc_counts['escalated'], color='#ff7f0e')
plt.title('Total Escalations per Patient')
plt.xlabel('Patient')
plt.ylabel('Escalations')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Step 6b — Interactive Charts (Plotly)
Interactively explore adherence trends and escalations.

In [ ]:
import plotly.express as px

# Interactive line chart per patient
fig = px.line(df, x='day', y='score', color='patient_name',
                markers=True, title='Adherence Score Over Days (Interactive)')
fig.update_layout(xaxis_title='Day', yaxis_title='Score (0-100)')
fig.show()

In [ ]:
# Interactive bar chart of escalations
fig2 = px.bar(df.groupby('patient_name', as_index=False)['escalated'].sum(),
                x='patient_name', y='escalated',
                title='Total Escalations per Patient (Interactive)',
                color='patient_name')
fig2.update_layout(xaxis_title='Patient', yaxis_title='Escalations')
fig2.show()